In [1]:
import os
import json
import librosa
import numpy as np


### Test Model h5

In [6]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('/Users/danielporras/Musica/music_x/best_model.h8')

In [6]:
from sklearn.preprocessing import LabelEncoder

# Assume 'labels' is a list of your labels
labels = ['Red', 'Blue', 'Green', 'Yellow', 'Black', 'Gray', 'Sky Blue', 'Orange']

encoder = LabelEncoder()
encoder.fit(labels)

print('Class labels:', encoder.classes_)

Class labels: ['Black' 'Blue' 'Gray' 'Green' 'Orange' 'Red' 'Sky Blue' 'Yellow']


In [7]:
def load_and_preprocess_songs(directory):
    song_features = []
    song_names = []  # List to store the names of the songs
    
    for filename in os.listdir(directory):
        if filename.endswith('.mp3') or filename.endswith('.m4a'):  # Assuming the songs are in .mp3 or .m4a format
            song_path = os.path.join(directory, filename)
            mfccs = extract_features(song_path)
            
            if mfccs is not None:  # Only append features if mfccs could be computed
                # Reshape the data to match the expected input shape
                mfccs = np.reshape(mfccs, (26, 1, 1))
                song_features.append(mfccs)
                song_names.append(filename)  # Store the name of the song
                
    # Convert list to numpy array
    song_features = np.array(song_features)

    # Ensure the data has the correct shape
    if len(song_features.shape) < 4:
        # Add a dimension for the batch size
        song_features = np.expand_dims(song_features, axis=0)

    return song_features, song_names

In [8]:
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast') 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=26)
    mfccs_processed = np.mean(mfccs.T,axis=0)
     
    return mfccs_processed

In [9]:
# Define the directory with the test songs
test_dir = '/Users/danielporras/Musica/music_import/test_songs'

# Load and preprocess the songs
test_songs, test_song_names = load_and_preprocess_songs(test_dir)

# Use the loaded model to make predictions
predictions = loaded_model.predict(test_songs)

# Get the class with the highest probability
predicted_classes = np.argmax(predictions, axis=1)

# Define the mapping from indices to class labels
class_labels = ['happy_exciting', 'heavy_rock', 'sad_reflective']

# Print the predicted classes along with the song names
for song_name, predicted_class in zip(test_song_names, predicted_classes):
    print(f'Song: {song_name}, Predicted class: {class_labels[predicted_class]}')

/var/folders/_t/p3lht79d2_jd6gwjl515j1g40000gn/T/ipykernel_98076/3396950221.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
/Users/danielporras/Musica/new_env_3_10/lib/python3.10/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


ValueError: in user code:

    File "/Users/danielporras/Musica/new_env_3_10/lib/python3.10/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/Users/danielporras/Musica/new_env_3_10/lib/python3.10/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/danielporras/Musica/new_env_3_10/lib/python3.10/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/danielporras/Musica/new_env_3_10/lib/python3.10/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/Users/danielporras/Musica/new_env_3_10/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/d

In [19]:
num_classes = loaded_model.output_shape[-1]
print(f'The model can predict {num_classes} different classes.')

The model can predict 10 different classes.


In [2]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('/Users/danielporras/Musica/music_x/best_model.h7')

### Test using scaler

In [18]:
import os
import json
import librosa
import numpy as np
from pydub import AudioSegment
import traceback

# Define the function to extract MFCC features from an audio file
def extract_features(file_path, num_segments=5, max_pad_length=174):
    try:
        audio = AudioSegment.from_file(file_path)
        audio = audio.set_frame_rate(44100).set_channels(1)
        audio.export("temp.wav", format="wav")
        y, sr = librosa.load('temp.wav', sr=44100)

        if len(y) < 30 * sr:
            y = np.pad(y, int(np.ceil((30 * sr - len(y)) / 2)), mode='reflect')
        y = y[int(len(y) / 2 - 15 * sr):int(len(y) / 2 + 15 * sr)]

        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        if mfccs.shape[1] > max_pad_length:
            mfccs = mfccs[:, :max_pad_length]
        else:
            padding = max_pad_length - mfccs.shape[1]
            mfccs = np.pad(mfccs, ((0, 0), (0, padding)), mode='constant')

        os.remove("temp.wav")
        return mfccs
    except Exception as e:
        print(f"Error encountered while parsing file: {file_path}")
        print(traceback.format_exc())
        return None

# Path to the directory containing the audio files for prediction
base_dir = '/Users/danielporras/Musica/musica_analysis/music_files/test_songs'
features, song_names = [], []

# Extract features for each audio file
for file_name in os.listdir(base_dir):
    if file_name.endswith('.mp3') or file_name.endswith('.m4a'):
        file_path = os.path.join(base_dir, file_name)
        file_features = extract_features(file_path)
        if file_features is not None:
            features.append(file_features.tolist())  # Convert numpy array to list for JSON serialization
            song_names.append(file_name)

if features:
    # Save the features and song names for later prediction
    data = {
        "features": features,
        "song_names": song_names
    }
    
    with open('data_unlabeled.json', 'w') as fp:
        json.dump(data, fp)
    print("Feature extraction and data saving completed.")
else:
    print("No features were extracted. Check the audio files and extraction process.")


Feature extraction and data saving completed.


In [19]:
import json
import numpy as np
import pickle
import tensorflow as tf

# Load the features from the JSON file
with open('/Users/danielporras/Musica/musica_analysis/models/data_unlabeled.json', 'r') as fp:
    data = json.load(fp)
features = np.array(data["features"])

# Load the scaler and transform the features
with open('/Users/danielporras/Musica/musica_analysis/models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
features_flattened = features.reshape(features.shape[0], -1)
features_normalized = scaler.transform(features_flattened)
features_normalized = features_normalized.reshape(features.shape)

# Load the model
model_path = '/Users/danielporras/Musica/music_x/best_model.h8'
model = tf.keras.models.load_model(model_path)

# Predict the classes of the songs
predictions_prob = model.predict(features_normalized)
predictions = np.argmax(predictions_prob, axis=1)

# Map predictions to class names
class_names = ['happy_exciting', 'heavy_rock', 'sad_reflective']
predicted_classes = [class_names[i] for i in predictions]

# Print the predicted class for each song
for song_name, predicted_class in zip(data["song_names"], predicted_classes):
    print(f"{song_name}: {predicted_class}")


2/2 [==============================] - 0s 2ms/step
watermarked_AEROPLANES_Groovy_Town_instrumental_2_31 copy.mp3: heavy_rock
watermarked_Music_City_Reggae_Squad_Steel_Pan_Island_instrumental_2_36.mp3: sad_reflective
Buscando-el-Son-no-voice_AdobeStock_709714291_preview copy.m4a: happy_exciting
watermarked_Music_City_Reggae_Squad_Steel_Pan_Island_instrumental_2_36 copy.mp3: sad_reflective
watermarked_LNDO_Just_Look_Up_background_vocals_2_45 copy.mp3: heavy_rock
watermarked_Jed_Stark_Holidaze_instrumental_2_03.mp3: heavy_rock
watermarked_Material_Gurl_The_Happiest_Memories_instrumental_1_56.mp3: heavy_rock
Spanish-Summer-Dance_AdobeStock_452607663_preview copy.m4a: happy_exciting
watermarked_Reveille_Go_For_It_instrumental_2_48 copy.mp3: heavy_rock
watermarked_Tiger_Gang_Hotpants_instrumental_1_20.mp3: heavy_rock
watermarked_PALA_Imagination_background_vocals_3_08 copy.mp3: sad_reflective
FEEL-GOOD-HAPPY-WORLD_AdobeStock_537914173_preview copy.m4a: happy_exciting
watermarked_Reveille_Go_